## Imports

In [ ]:
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import mido
import librosa
import librosa.display
from IPython.display import Audio as ipy_audio

## Read Sample

In [ ]:
mid = mido.MidiFile("midi-samples/Bahar.mid")

In [ ]:
track = mid.tracks[0]

# # meta
# for ii, mm in enumerate(track):
#     if type(mm) != mido.messages.messages.Message:
#         print(mm)

# # other signals
# for ii, mm in enumerate(track):
#     if type(mm) == mido.messages.messages.Message:
#         if mm.type != "note_on":
#             print(mm)

track_notes = []
for ii, mm in enumerate(track):
    if type(mm) == mido.messages.messages.Message:
        if mm.type == "note_on":
            track_notes.append(mm)

In [ ]:
# track_notes

In [ ]:
midi_length = mid.length
midi_ticks_per_beat = mid.ticks_per_beat

## Convert to Audio

In [ ]:
# Constants
sr = 22050  # Sampling rate for audio
velocity_scaling = 127.0  # Maximum MIDI velocity
note_on_velocity_threshold = 0  # Threshold to consider a note_on event as actually on

In [ ]:
# Helper function to convert MIDI note to frequency
def midi_to_freq(midi_note):
    return 440.0 * 2**((midi_note - 69) / 12.0)

In [ ]:
# Create a blank audio array
audio_length = int(mido.tick2second(midi_length, midi_ticks_per_beat, 500000) * sr)
audio = np.zeros(audio_length*sr)
# Create a blank symbolic array
# TODO(neeraja): take notes equi-spaced in time
nts = np.array([0,]*len(track_notes))


# Time tracking
current_time = 0

for ii, msg in enumerate(track_notes):
    assert msg.type == "note_on"
    nts[ii] = msg.note
    
    # Calculate time in seconds and samples
    note_time = mido.tick2second(msg.time, midi_ticks_per_beat, 500000)
    note_samples = int(note_time * sr)

    # Calculate the start and end sample indices
    start_sample = current_time
    end_sample = current_time + note_samples

    if msg.velocity >= note_on_velocity_threshold:  # Note on event
        freq = midi_to_freq(msg.note)
        duration = (note_samples / sr)

        # Generate a sine wave for the note
        t = np.linspace(0, duration, note_samples, False)
        wave = 0.5 * np.sin(2 * np.pi * freq * t) * (msg.velocity / velocity_scaling)

        # Add the generated wave to the audio array
        assert len(wave) == end_sample - start_sample
        audio[start_sample:start_sample+len(wave)] += wave

    # Move current time forward
    current_time = end_sample

# Normalize the audio to avoid clipping
audio = audio / np.max(np.abs(audio))


In [ ]:
ipy_audio(data=audio[:1500000], rate=sr)

In [ ]:
root = 49

## Symbol String

In [ ]:
def to_pitch_class(note):
    return librosa.midi_to_svara_h(note, Sa=root, abbr=True, octave=False, unicode=False)

In [ ]:
syms = [to_pitch_class(nn) for nn in nts]

## Markov Analysis